In [1]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.9 MB/s eta 0:00:00


In [2]:
!pip install lmoments3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.5 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import scipy.stats as st
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import anderson
from scipy.stats import kstest
from scipy.stats import jarque_bera
from statsmodels.stats.diagnostic import lilliefors
from scipy.stats import normaltest  #D`Agostino
from scipy.stats import chisquare
import seaborn as sns
from sklearn import tree
from scipy.stats import differential_entropy, norm
import statistics
import scipy.special
from catboost import CatBoostClassifier
import lmoments3 as lm
import math

In [9]:
def FeatureCompute(vec):
  feature=[]
  feature.append(np.mean(vec))
  feature.append(st.kurtosis(vec))
  feature.append(st.skew(vec))
  feature.append(np.quantile(vec, 0.125))
  feature.append(np.quantile(vec, 0.25))
  feature.append(np.quantile(vec, 0.375))
  feature.append(np.quantile(vec, 0.625))
  feature.append(np.quantile(vec, 0.75))
  feature.append(np.quantile(vec,0.875))
  feature.append(differential_entropy(vec))
  feature.append(len(vec))
  feature.append(np.min(vec))
  feature.append(np.max(vec))

  l_1,l_2,l_skewness,l_kurtosis=lm.lmom_ratios(vec,nmom=4)
  feature.append(l_1)
  feature.append(l_2)
  feature.append(l_skewness)
  feature.append(l_kurtosis)

  return feature

In [11]:
def TrainSet(Distribution_Id, SampleSize, repeat):
  myDataSet=[]
  myLabels=[]
  for k in range(len(Distribution_Id)):
    for j in range(len(SampleSize)):
      for i in range(repeat[k][j]):
        if Distribution_Id[k]==0: myData = list(np.random.normal(size=SampleSize[j]))
        if Distribution_Id[k]==1: myData = list(np.random.beta(2,2, size=SampleSize[j]))
        if Distribution_Id[k]==2: myData = list(np.random.logistic(0,1,size=SampleSize[j]))
        if Distribution_Id[k]==3: myData = list(np.random.standard_t(5,size=SampleSize[j]))
        if Distribution_Id[k]==4: myData = list(np.random.uniform(0,1,size=SampleSize[j]))
        if Distribution_Id[k]==5: myData = list(np.random.laplace(size=SampleSize[j]))
        if Distribution_Id[k]==6: myData =list(np.random.beta(2,5, size=SampleSize[j]))
        if Distribution_Id[k]==7: myData = list(np.random.gamma(4,5, size=SampleSize[j]))
        if Distribution_Id[k]==8: myData =list(np.random.lognormal(size=SampleSize[j]))
        if Distribution_Id[k]==9: myData = list(np.random.chisquare(5, size=SampleSize[j]))
        if Distribution_Id[k]==10: myData = list(np.random.exponential(size=SampleSize[j]))

        myDataStd=(myData-np.mean(myData))/np.std(myData)
        myDataSort=np.sort(myDataStd)
        myFeatures = FeatureCompute(myDataSort)
        myDataSet.append(myFeatures)
        if Distribution_Id[k]==0:
          myLabels.append(1)
        else:
          myLabels.append(0)
  return myDataSet,myLabels

In [12]:
def TestSet(Distribution_Id, SampleSize, repeat):
  myDataSet=[]
  myLabels=[]
  ll_Results=[]
  ad_Results=[]
  dap_Results=[]
  jb_Results=[]
  sw_Results=[]

  for k in range(len(Distribution_Id)):
    for j in SampleSize:
      for i in range(repeat[k]):
        if Distribution_Id[k]==0: myData = list(np.random.normal(size=j))
        if Distribution_Id[k]==1: myData = list(np.random.beta(2,2, size=j))
        if Distribution_Id[k]==2: myData = list(np.random.logistic(0,1,size=j))
        if Distribution_Id[k]==3: myData = list(np.random.standard_t(5,size=j))
        if Distribution_Id[k]==4: myData = list(np.random.uniform(0,1,size=j))
        if Distribution_Id[k]==5: myData = list(np.random.laplace(size=j))
        if Distribution_Id[k]==6: myData =list(np.random.beta(2,5, size=j))
        if Distribution_Id[k]==7: myData = list(np.random.gamma(4,5, size=j))
        if Distribution_Id[k]==8: myData =list(np.random.lognormal(size=j))
        if Distribution_Id[k]==9: myData = list(np.random.chisquare(5, size=j))
        if Distribution_Id[k]==10: myData = list(np.random.exponential(size=j))

        myDataStd=(myData-np.mean(myData))/np.std(myData)
        myDataSort=np.sort(myDataStd)
        myFeatures = FeatureCompute(myDataSort)
        myDataSet.append(myFeatures)
        if Distribution_Id[k]==0:
          myLabels.append(1)
        else:
          myLabels.append(0)

        stat_ll, ll_p_value = lilliefors(myDataSort)
        if ll_p_value < 0.05:
          ll_Results.append(1)
        else:
          ll_Results.append(0)

        result = anderson(myDataSort,dist="norm")
        if result[1][2]<result[0]:
          ad_Results.append(1)
        else:
          ad_Results.append(0)

        stat_dap, dap_p_value = normaltest(myDataSort)
        if dap_p_value < 0.05:
          dap_Results.append(1)
        else:
          dap_Results.append(0)

        stat_jb, jb_p_value = jarque_bera(myDataSort)
        if jb_p_value < 0.05:
          jb_Results.append(1)
        else:
          jb_Results.append(0)

        stat_sw, sw_p_value = shapiro(myDataSort)
        if sw_p_value < 0.05:
          sw_Results.append(1)
        else:
          sw_Results.append(0)

  return myDataSet,myLabels,ll_Results, ad_Results, dap_Results, jb_Results, sw_Results

In [15]:
SampleSize=[10,20,30,40,50,100,500]
repeat=[[3500,2800,2800,1600,1600,1000,100],
        [10,10,10,10,10,20,30],
        [700,550,350,250,200,100,100],
        [600,550,550,250,200,200,100],
        [10,10,10,20,30,30,10],
        [200,30,30,30,30,30,30],
        [10,10,10,10,10,30,30],
        [10,10,10,10,10,20,10],
        [20,20,20,10,10,10,10],
        [10,10,10,20,20,20,10],
        [10,20,20,20,20,10,10]]

Distribution_Id=[0,1,2,3,4,5,6,7,8,9,10]

X, y = TrainSet(Distribution_Id=Distribution_Id,
                SampleSize=SampleSize,
                repeat=repeat)

xN= pd.DataFrame(X)

ctb = CatBoostClassifier(iterations=100)
ctb_model = ctb.fit(X,y)

Learning rate set to 0.300541
0:	learn: 0.6192106	total: 32.6ms	remaining: 3.23s
1:	learn: 0.5788087	total: 52.5ms	remaining: 2.57s
2:	learn: 0.5544945	total: 74ms	remaining: 2.39s
3:	learn: 0.5399606	total: 91.3ms	remaining: 2.19s
4:	learn: 0.5290589	total: 110ms	remaining: 2.09s
5:	learn: 0.5217339	total: 128ms	remaining: 2s
6:	learn: 0.5173114	total: 146ms	remaining: 1.94s
7:	learn: 0.5141588	total: 164ms	remaining: 1.88s
8:	learn: 0.5116618	total: 181ms	remaining: 1.83s
9:	learn: 0.5100997	total: 205ms	remaining: 1.84s
10:	learn: 0.5074157	total: 227ms	remaining: 1.83s
11:	learn: 0.5058553	total: 250ms	remaining: 1.83s
12:	learn: 0.5043528	total: 257ms	remaining: 1.72s
13:	learn: 0.5035560	total: 263ms	remaining: 1.62s
14:	learn: 0.5028479	total: 270ms	remaining: 1.53s
15:	learn: 0.5016682	total: 277ms	remaining: 1.45s
16:	learn: 0.5005544	total: 284ms	remaining: 1.39s
17:	learn: 0.4996948	total: 291ms	remaining: 1.33s
18:	learn: 0.4990813	total: 297ms	remaining: 1.27s
19:	learn: 0

In [20]:
TestSampleSize=[10,20,30,40,50,100,500]
n_simulations=[10]
TestDistribution_Id=[2]
testRepeat=100


for n in TestSampleSize:
  myPerformances=[]
  ll_Performances=[]
  ad_Performances=[]
  dap_Performances=[]
  jb_Performances=[]
  sw_Performances=[]
  for m in range(testRepeat):
    testX, testY, ll, ad, dap, jb, sw = TestSet(Distribution_Id=TestDistribution_Id,SampleSize=[n],repeat=n_simulations)
    xN= pd.DataFrame(testX)
    predictedY = ctb.predict(xN)
    acc= sum(predictedY==testY)/len(predictedY)
    myPerformances.append(acc)

    acc_ll=sum(ll)/len(ll)
    acc_ad=sum(ad)/len(ad)
    acc_dap=sum(dap)/len(dap)
    acc_jb=sum(jb)/len(jb)
    acc_sw=sum(sw)/len(sw)

    ll_Performances.append(acc_ll)
    ad_Performances.append(acc_ad)
    dap_Performances.append(acc_dap)
    jb_Performances.append(acc_jb)
    sw_Performances.append(acc_sw)

  if TestDistribution_Id[0]==0:
    error=np.round(1-np.mean(myPerformances),5)
    print('Sample Size n=',n,'---> MLPN Type I error rate = ' ,error )
    print('Sample Size n=',n,'---> The LL Test Type I error rate = ' , np.round(np.mean(ll_Performances),5))
    print('Sample Size n=',n,'---> The AD Test Type I error rate = ' , np.round(np.mean(ad_Performances),5))
    print('Sample Size n=',n,'---> The DAP Test Type I error rate = ' , np.round(np.mean(dap_Performances),5))
    print('Sample Size n=',n,'---> The JB Test Type I error rate = ' , np.round(np.mean(jb_Performances),5))
    print('Sample Size n=',n,'---> The SW Test Type I error rate = ' , np.round(np.mean(sw_Performances),5),'\n')
  else:
    print('Sample Size n=',n,'---> Power of MLPN = ' , np.round(np.mean(myPerformances),5))
    print('Sample Size n=',n,'---> Power of The LL test = ' , np.round(np.mean(ll_Performances),5))
    print('Sample Size n=',n,'---> Power of The AD test = ' , np.round(np.mean(ad_Performances),5))
    print('Sample Size n=',n,'---> Power of The DAP test = ' , np.round(np.mean(dap_Performances),5))
    print('Sample Size n=',n,'---> Power of The JB test = ' , np.round(np.mean(jb_Performances),5))
    print('Sample Size n=',n,'---> Power of The SW test = ' , np.round(np.mean(sw_Performances),5),'\n')


Sample Size n= 10 ---> Power of MLPN =  0.057
Sample Size n= 10 ---> Power of The LL test =  0.064
Sample Size n= 10 ---> Power of The AD test =  0.07
Sample Size n= 10 ---> Power of The DAP test =  0.097
Sample Size n= 10 ---> Power of The JB test =  0.023
Sample Size n= 10 ---> Power of The SW test =  0.067 

Sample Size n= 20 ---> Power of MLPN =  0.108
Sample Size n= 20 ---> Power of The LL test =  0.08
Sample Size n= 20 ---> Power of The AD test =  0.109
Sample Size n= 20 ---> Power of The DAP test =  0.142
Sample Size n= 20 ---> Power of The JB test =  0.075
Sample Size n= 20 ---> Power of The SW test =  0.107 

Sample Size n= 30 ---> Power of MLPN =  0.17
Sample Size n= 30 ---> Power of The LL test =  0.085
Sample Size n= 30 ---> Power of The AD test =  0.125
Sample Size n= 30 ---> Power of The DAP test =  0.169
Sample Size n= 30 ---> Power of The JB test =  0.124
Sample Size n= 30 ---> Power of The SW test =  0.122 

Sample Size n= 40 ---> Power of MLPN =  0.221
Sample Size n= 